# 智能体框架 Framework

照例先感谢Datawhale的课程：https://datawhalechina.github.io/hello-agents/#/./chapter6/%E7%AC%AC%E5%85%AD%E7%AB%A0%20%E6%A1%86%E6%9E%B6%E5%BC%80%E5%8F%91%E5%AE%9E%E8%B7%B5

课程介绍了四种主流的智能体框架：

- Autogen

- AgentScope

- CAMEL

- LangGraph

每个框架都有自己优缺点，教程里都有，不再复述。重点还是实践。

从自己最感兴趣的开始，首先是LangGraph。

## LangGraph

最重要的概念是graph，图。图由节点（node）和边（edge）组成。节点表示任务或操作，边表示节点之间的关系或数据流。

Node我就简单粗暴地理解为函数(Function)，而state是函数的输入输出。到了某个edge（END）时，表示图的执行结束，输出结果。

Graph是某种由所有参与方都能看到的共享知识库（shared knowledge base），所有节点和边的信息都存储在图中，供后续节点查询和使用。

挺抽象的，和chatGPT学习了几轮之后，一起完成了下面的代码。

比较简单的流程：

1. 用户输入一个主题(topic)。
2. 第一个节点(NODE 1)生成与主题的大纲(Outline)。
3. 第二个节点(NODE 2)根据大纲生成详细内容(draft)。
4. 第三个节点(NODE 3)对内容进行润色(refine），指定风格。

实际测试了几轮下来，英文输出比中文好很多，可能是gpt本身的语言能力+我本身习惯用英文提示词？

流程完全没问题，挺有意思的。感觉就是把AI给串在一起了。

如果是特别简单的应用，应该不依赖框架，也能直接通过输入-输出-输入这样的流动串起来。但是框架给了建造复杂智能体的可能性，等课程学完了可以好好琢磨下。

这个agent的课程比往期课程的强度要大，感觉三天时间好紧张，没办法一一完成测试和学习了，只能挑挑拣拣，按照自己的能力和兴趣来学了。

In [7]:
import os
from openai import OpenAI
from colorama import Fore, Style

from camel.societies import RolePlaying
from camel.utils import print_text_animated

# ========= 你的初始化（原封不动） =========
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not set in environment. Please set it before running this notebook.")

# 注意：你强调的 base url
base_url = os.getenv("OPENAI_API_BASE", "https://xiaoai.plus/v1")
model_name = os.getenv("OPENAI_MODEL", "gpt-5-mini")

client = OpenAI(api_key=api_key, base_url=base_url)

process_log = []

# ========= 只做“适配”，不重建 client =========
# 不同 camel-ai 版本导入路径可能略有差异，这里做兼容导入
try:
    from camel.models import OpenAIModel
except Exception:
    from camel.models.openai_model import OpenAIModel

# 这里不需要你再配置 base_url
# base_url 已经在 client 里了
camel_model = OpenAIModel(
    model_type=model_name,
    client=client,                # 关键：注入你现成的 client
    model_config_dict=None,       # 关键：不额外配一堆 config，走默认
)

# ========= CAMEL 角色设定 =========
assistant_role = "无厘头科普写手"
user_role = "科学审稿人"

assistant_prompt = (
    "你负责写无厘头科普，允许夸张和离谱比喻。"
    "只能在表达上胡来，事实必须正确。"
    "输出中文。"
)

user_prompt = (
    "你是严格科学审稿人。"
    "指出事实错误、含混和可能误导的表达，并给出明确可执行的改写要求。"
    "输出中文，用条目列出。"
)

# Compatibility wrapper for RolePlaying constructor
def build_role_playing_compatible(**kwargs):
    """Try multiple common kwarg-name variants for RolePlaying constructor."""
    import inspect, traceback
    attempts = [
        # variant used in some examples
        ["assistant_role_name","user_role_name","assistant_role_prompt","user_role_prompt","model"],
        # some versions use assistant_prompt/user_prompt
        ["assistant_role_name","user_role_name","assistant_prompt","user_prompt","model"],
        # some versions expect assistant_role and user_role dicts
        ["assistant_role","user_role","model"],
        # some possible shortened names
        ["assistant_name","user_name","assistant_prompt","user_prompt","model"],
        # bare minimal
        ["assistant_role_name","user_role_name","model"],
    ]

    last_exc = None
    for keys in attempts:
        kwargs_try = {k: v for k, v in kwargs.items() if k in keys}
        try:
            rp = RolePlaying(**kwargs_try)
            print("RolePlaying succeeded with keys:", sorted(list(kwargs_try.keys())))
            return rp
        except TypeError as e:
            last_exc = e
        except Exception as e:
            last_exc = e
            traceback.print_exc()

    # If not successful, raise informative error with signature
    sig = inspect.signature(RolePlaying.__init__)
    raise TypeError(f"All attempts failed. Last exception: {last_exc}.\nRolePlaying.__init__ signature: {sig}")

# Build role_playing using compatibility wrapper
role_playing = build_role_playing_compatible(
    assistant_role_name=assistant_role,
    user_role_name=user_role,
    assistant_role_prompt=assistant_prompt,
    user_role_prompt=user_prompt,
    model=camel_model,
)

# ========= 启动对话 =========
topic = "为什么人会打喷嚏"
initial_prompt = f"主题：{topic}\n请写一段 300 到 400 字的无厘头科普草稿。"

assistant_msg, user_msg = role_playing.init_chat(initial_prompt)

print_text_animated(Fore.GREEN + "=== 初稿（无厘头写手） ===\n" + Style.RESET_ALL)
print_text_animated(assistant_msg.content)

# 记录日志，方便你学习回看
process_log.append({"stage": "draft", "text": assistant_msg.content})

# ========= 多轮审稿与重写 =========
rounds = 3
for i in range(rounds):
    print_text_animated(Fore.CYAN + f"\n=== 第 {i+1} 轮：科学审稿 ===\n" + Style.RESET_ALL)
    assistant_msg, user_msg = role_playing.step(assistant_msg)
    print_text_animated(user_msg.content)
    process_log.append({"stage": f"review_{i+1}", "text": user_msg.content})

    print_text_animated(Fore.GREEN + f"\n=== 第 {i+1} 轮：无厘头重写 ===\n" + Style.RESET_ALL)
    assistant_msg, user_msg = role_playing.step(user_msg)
    print_text_animated(assistant_msg.content)
    process_log.append({"stage": f"rewrite_{i+1}", "text": assistant_msg.content})

print_text_animated(Fore.YELLOW + "\n=== 最终版本（可发布） ===\n" + Style.RESET_ALL)
print_text_animated(assistant_msg.content)
process_log.append({"stage": "final", "text": assistant_msg.content})
    display(Markdown("**Draft (stored to state):**\n\n" + ("```text\n" + draft + "\n```" if draft else "(empty)")))

    return {"draft": draft}


def refine_draft(state: ArticleState) -> ArticleState:
    """Refine the existing draft into a 'New Scientist' style: concise, engaging, lightly journalistic but scientifically grounded."""
    draft = state.get("draft", "")
    outline = state.get("outline", "")
    display(Markdown("### Node: refine_draft — refining draft into ‘New Scientist’ style"))

    prompt = (
        "You are an editor for a popular science magazine (New Scientist style).\n"
        "Rewrite the provided draft to be concise, engaging, and accessible to a general audience.\n"
        "Keep scientific accuracy, use clear explanations and a slightly journalistic tone, and avoid jargon where possible.\n"
        "Limit the refined draft to about 150-250 words.\n\n"
        f"Outline:\n{outline}\n\nDraft:\n{draft}\n\n"
        "Return only the refined article text (no extra commentary)."
    )

    refined = call_llm(prompt)

    display(Markdown("**Refined draft (stored to state):**\n\n" + ("```text\n" + refined + "\n```" if refined else "(empty)")))

    return {"refined_draft": refined}


# 3️⃣ 构建 Graph
def build_graph():
    graph = StateGraph(ArticleState)

    graph.add_node("outline", make_outline)
    graph.add_node("draft", write_draft)
    graph.add_node("refine", refine_draft)

    graph.set_entry_point("outline")
    graph.add_edge("outline", "draft")
    graph.add_edge("draft", "refine")
    graph.add_edge("refine", END)

    return graph.compile()


# 4️⃣ 运行（示例）
if __name__ == "__main__":
    app = build_graph()

    initial_state = {"topic": "elderly smartphone usage and health"}

    display(Markdown("## Running the agent graph"))
    display(Markdown("**Initial state:**\n\n```json\n" + str(initial_state) + "\n```"))

    final_state = app.invoke(initial_state)

    # Present final state as Markdown for better readability
    display(Markdown("## Final state (summary)"))
    topic = final_state.get("topic", initial_state.get("topic", "(none)"))
    outline = final_state.get("outline", "(none)")
    draft = final_state.get("draft", "(none)")
    refined = final_state.get("refined_draft", "(none)")

    display(Markdown(f"### Topic:\n\n**{topic}**"))
    display(Markdown("### Outline:\n\n" + ("```text\n" + outline + "\n```" if outline else "(none)")))
    display(Markdown("### Draft:\n\n" + ("```text\n" + draft + "\n```" if draft else "(none)")))
    display(Markdown("### Refined Draft (New Scientist style):\n\n" + ("```text\n" + refined + "\n```" if refined else "(none)")))

    # Optionally show the process log as an ordered list
    display(Markdown("## Process log (LLM calls in order)"))
    for i, entry in enumerate(process_log, 1):
        resp_len = len(entry.get('response') or "")
        display(Markdown(f"**Step {i}** — stored response length: {resp_len} chars"))

    display(Markdown("---\nRun complete."))


## Running the agent graph

**Initial state:**

```json
{'topic': 'elderly smartphone usage and health'}
```

### Node: make_outline — topic: **elderly smartphone usage and health**

**LLM prompt:**
```text
You are an assistant that writes short article outlines.
Produce a concise outline (3-5 bullet points) for an article about: elderly smartphone usage and health
Return the outline as plain text, each bullet on a new line.
```

**LLM response:**
```text
- Benefits: how smartphones support seniors through social connection, telehealth, medication reminders, and emergency alerts.

- Health risks and mitigations: address eye strain, sleep disruption, and posture issues with accessibility settings, blue-light filters, larger text, and regular breaks.

- Practical guidelines for adoption: recommend senior-friendly devices and apps, voice assistants, clear onboarding, and ongoing tech support with caregiver involvement.

- Safety, privacy, and support: emphasize data privacy, scam awareness, built-in safety features (fall detection, SOS), and access to local training resources.
```

...

## Process log (LLM calls in order)

**Step 1** — stored response length: 630 chars

**Step 2** — stored response length: 1551 chars

**Step 3** — stored response length: 1442 chars

---
Run complete.

In [ ]:
import os
from openai import OpenAI
from colorama import Fore, Style

# Try to import RolePlaying and helper; different camel versions may expose paths differently
try:
    from camel.societies import RolePlaying
    from camel.utils import print_text_animated
except Exception:
    # re-raise a clearer error if import fails
    raise

# ========= 你的初始化（原封不动） =========
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not set in environment. Please set it before running this notebook.")

# 注意：你强调的 base url
base_url = os.getenv("OPENAI_API_BASE", "https://xiaoai.plus/v1")
model_name = os.getenv("OPENAI_MODEL", "gpt-5-mini")

# reuse existing OpenAI client if present; else construct a minimal one
try:
    client
except NameError:
    client = OpenAI(api_key=api_key, base_url=base_url)

process_log = []

# If camel provides OpenAIModel, try to wrap the client; otherwise we'll pass the model name directly
camel_model = None
try:
    try:
        from camel.models import OpenAIModel
    except Exception:
        from camel.models.openai_model import OpenAIModel
    camel_model = OpenAIModel(model_type=model_name, client=client, model_config_dict=None)
except Exception:
    # Not fatal; we'll attempt to pass model_name directly to RolePlaying if accepted
    camel_model = None

# ========= CAMEL 角色设定 =========
assistant_role = "无厘头科普写手"
user_role = "科学审稿人"

assistant_prompt = (
    "你负责写无厘头科普，允许夸张和离谱比喻。"
    "只能在表达上胡来，事实必须正确。"
    "输出中文。"
)

user_prompt = (
    "你是严格科学审稿人。"
    "指出事实错误、含混和可能误导的表达，并给出明确可执行的改写要求。"
    "输出中文，用条目列出。"
)

# Robust RolePlaying construction: try a few common signatures without passing unsupported keys
role_playing = None
last_exc = None

# Attempt 1: common named args (some versions accept assistant_role_name + assistant_prompt)
try:
    kwargs = {
        "assistant_role_name": assistant_role,
        "user_role_name": user_role,
        "assistant_prompt": assistant_prompt,
        "user_prompt": user_prompt,
    }
    if camel_model is not None:
        kwargs["model"] = camel_model
    else:
        kwargs["model"] = model_name
    role_playing = RolePlaying(**kwargs)
    print("RolePlaying instantiated with assistant_prompt/user_prompt variant")
except TypeError as e:
    last_exc = e

# Attempt 2: some versions expect assistant_role_name + user_role_name + model only
if role_playing is None:
    try:
        kwargs = {
            "assistant_role_name": assistant_role,
            "user_role_name": user_role,
        }
        if camel_model is not None:
            kwargs["model"] = camel_model
        else:
            kwargs["model"] = model_name
        role_playing = RolePlaying(**kwargs)
        print("RolePlaying instantiated with assistant_role_name/user_role_name + model")
    except TypeError as e:
        last_exc = e

# Attempt 3: dict-style roles (some versions use assistant_role/user_role dicts)
if role_playing is None:
    try:
        assistant_role_dict = {"name": assistant_role, "prompt": assistant_prompt}
        user_role_dict = {"name": user_role, "prompt": user_prompt}
        kwargs = {"assistant_role": assistant_role_dict, "user_role": user_role_dict}
        if camel_model is not None:
            kwargs["model"] = camel_model
        else:
            kwargs["model"] = model_name
        role_playing = RolePlaying(**kwargs)
        print("RolePlaying instantiated with assistant_role/user_role dicts")
    except TypeError as e:
        last_exc = e

# Attempt 4: try minimal bare model-only init
if role_playing is None:
    try:
        role_playing = RolePlaying(model=camel_model if camel_model is not None else model_name)
        print("RolePlaying instantiated with model-only argument")
    except Exception as e:
        last_exc = e

if role_playing is None:
    # Provide a helpful error message including the last exception
    raise TypeError(f"Failed to construct RolePlaying. Last exception: {last_exc}")

# ========= 启动对话 =========
topic = "为什么人会打喷嚏"
initial_prompt = f"主题：{topic}\n请写一段 300 到 400 字的无厘头科普草稿。"

# Helpers to extract plain text from various return objects
import types

def _get_content(obj):
    """Return a text string extracted from obj. Works for str, objects with .content, dicts, and camel ChatAgentResponse-like objects with .msgs."""
    if obj is None:
        return None
    if isinstance(obj, str):
        return obj
    # objects with .content
    if hasattr(obj, "content") and isinstance(obj.content, str):
        return obj.content
    # dict-like
    if isinstance(obj, dict):
        for key in ("content", "text", "message", "msg"):
            if key in obj and isinstance(obj[key], str):
                return obj[key]
        return str(obj)
    # camel ChatAgentResponse has .msgs (list); each item may have .content
    if hasattr(obj, "msgs") and isinstance(obj.msgs, (list, tuple)):
        for m in obj.msgs:
            if hasattr(m, "content") and isinstance(m.content, str):
                return m.content
        # fallback to stringifying first element
        try:
            return str(obj.msgs[0])
        except Exception:
            return str(obj)
    # nested attribute 'message'
    if hasattr(obj, "message") and hasattr(obj.message, "content"):
        return obj.message.content
    # fallback
    try:
        return str(obj)
    except Exception:
        return None


def _normalize_pair_to_contents(result):
    """Return tuple of (assistant_text, user_text) both plain strings (either may be None).
    Accepts: tuple/list, single message-like object, dict, or object with msgs.
    """
    # tuple/list
    if isinstance(result, (tuple, list)):
        if len(result) >= 2:
            return _get_content(result[0]), _get_content(result[1])
        elif len(result) == 1:
            return _get_content(result[0]), None
    # single
    return _get_content(result), None


# Call init_chat and extract plain text
res = role_playing.init_chat(initial_prompt)
# --- Diagnostic block: inspect raw response and detect obvious duplication patterns ---
import unicodedata
# show a compact repr and type for the raw response object (may be Message, tuple, dict, etc.)
print('DIAGNOSTIC: type(res)=', type(res))
try:
    print('DIAGNOSTIC: repr(res)[:500]=', repr(res)[:500])
except Exception as _e:
    print('DIAGNOSTIC: repr(res) failed:', _e)
# if the object has msgs/content, show a bit more detail
if hasattr(res, 'msgs'):
    try:
        print('DIAGNOSTIC: res.msgs length=', len(res.msgs))
        for i, m in enumerate(res.msgs[:3]):
            print(f' DIAGNOSTIC: msg[{i}] type={type(m)}')
            if hasattr(m, 'content'):
                print('   DIAGNOSTIC: msg.content repr[:200]=', repr(m.content)[:200])
    except Exception as _e:
        print('DIAGNOSTIC: inspecting res.msgs failed:', _e)
if hasattr(res, 'content'):
    try:
        print('DIAGNOSTIC: res.content repr[:500]=', repr(res.content)[:500])
    except Exception as _e:
        print('DIAGNOSTIC: inspecting res.content failed:', _e)
# Normalize/extract plain strings into assistant_text/user_text using existing helper
assistant_text, user_text = _normalize_pair_to_contents(res)
# More diagnostics on the extracted assistant_text (if any)
if assistant_text is None:
    print('DIAGNOSTIC: assistant_text is None')
else:
    print('DIAGNOSTIC: type(assistant_text)=', type(assistant_text))
    try:
        print('DIAGNOSTIC: len(assistant_text)=', len(assistant_text))
        print('DIAGNOSTIC: repr(assistant_text)[:500]=', repr(assistant_text)[:500])
        print('DIAGNOSTIC: first 200 chars=')
        print(assistant_text[:200])
        print('DIAGNOSTIC: ords (first 80)=', list(map(ord, assistant_text[:80])))
    except Exception as _e:
        print('DIAGNOSTIC: assistant_text inspection failed:', _e)
# quick helper: detect if every character appears doubled (aa bb cc -> aabbcc pattern)
def is_every_char_duplicated(s, check_pairs=500):
    if not s:
        return False
    pairs = min(check_pairs, len(s)//2)
    if pairs == 0:
        return False
    for i in range(0, pairs*2, 2):
        if s[i] != s[i+1]:
            return False
    return True
# safe dedupe helper: if every-char duplicated use s[::2], else collapse runs to single chars (lossy but conservative)
def dedupe_if_obvious_double(s):
    if is_every_char_duplicated(s, check_pairs=500):
        return s[::2]
    # collapse runs: keep first of each run (avoids regex/backslash escaping issues in notebook JSON)
    out = []
    prev = None
    for ch in s:
        if ch != prev:
            out.append(ch)
            prev = ch
    return ''.join(out)
# check pattern and show a sample of the deduped output (do NOT modify original text yet)
if assistant_text:
    duplicated = is_every_char_duplicated(assistant_text, check_pairs=500)
    print('DIAGNOSTIC: every-char-duplicated (sample)=', duplicated)
    if duplicated:
        print('DIAGNOSTIC: sample deduped (first 400 chars)=')
        print(dedupe_if_obvious_double(assistant_text)[:400])
    else:
        print('DIAGNOSTIC: no obvious every-char duplication detected in sample')

# Print assistant initial draft if available
if assistant_text:
    print_text_animated(Fore.GREEN + "=== 初稿（无厘头写手） ===\n" + Style.RESET_ALL)
    print_text_animated(assistant_text)
    process_log.append({"stage": "draft", "text": assistant_text})
else:
    print_text_animated(Fore.YELLOW + "(No assistant initial message returned)" + Style.RESET_ALL)

# ========= 多轮审稿与重写 =========
rounds = 3
for i in range(rounds):
    # REVIEW: pass plain text into step to avoid MemoryRecord validation errors
    prev_for_review_text = assistant_text if assistant_text else user_text
    res = role_playing.step(prev_for_review_text)
    assistant_text_r, user_text_r = _normalize_pair_to_contents(res)

    # If a user_text was returned, treat that as the review output to print
    if user_text_r:
        print_text_animated(Fore.CYAN + f"\n=== 第 {i+1} 轮：科学审稿 ===\n" + Style.RESET_ALL)
        print_text_animated(user_text_r)
        process_log.append({"stage": f"review_{i+1}", "text": user_text_r})
    else:
        # If only assistant-like message returned, print it as reviewer output (best-effort)
        if assistant_text_r:
            print_text_animated(Fore.CYAN + f"\n=== 第 {i+1} 轮：科学审稿 (single-message) ===\n" + Style.RESET_ALL)
            print_text_animated(assistant_text_r)
            process_log.append({"stage": f"review_{i+1}", "text": assistant_text_r})

    # For rewrite, call step with the most recent user_text if present, else assistant_text
    prev_for_rewrite_text = user_text_r if user_text_r else assistant_text_r if assistant_text_r else (user_text or assistant_text)
    res2 = role_playing.step(prev_for_rewrite_text)
    assistant_text2, user_text2 = _normalize_pair_to_contents(res2)

    if assistant_text2:
        print_text_animated(Fore.GREEN + f"\n=== 第 {i+1} 轮：无厘头重写 ===\n" + Style.RESET_ALL)
        print_text_animated(assistant_text2)
        process_log.append({"stage": f"rewrite_{i+1}", "text": assistant_text2})

    # update the main assistant_text/user_text for next round
    assistant_text, user_text = assistant_text2 or assistant_text_r or assistant_text, user_text2 or user_text_r or user_text

print_text_animated(Fore.YELLOW + "\n=== 最终版本（可发布） ===\n" + Style.RESET_ALL)
if assistant_text:
    print_text_animated(assistant_text)
    process_log.append({"stage": "final", "text": assistant_text})
else:
    print_text_animated(Fore.YELLOW + "(No final assistant message)" + Style.RESET_ALL)


2025-12-30 22:23:12,210 - camel.societies.role_playing - WARNING - Model provided globally is set for all agents if not already specified in agent_kwargs.
RolePlaying instantiated with assistant_role_name/user_role_name + model
DIAGNOSTIC: type(res)= <class 'camel.messages.base.BaseMessage'>
DIAGNOSTIC: repr(res)[:500]= BaseMessage(role_name='无厘头科普写手', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict=None, content='主题：为什么人会打喷嚏\n请写一段 300 到 400 字的无厘头科普草稿。', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None, reasoning_content=None)
DIAGNOSTIC: res.content repr[:500]= '主题：为什么人会打喷嚏\n请写一段 300 到 400 字的无厘头科普草稿。'
DIAGNOSTIC: type(assistant_text)= <class 'str'>
DIAGNOSTIC: len(assistant_text)= 37
DIAGNOSTIC: repr(assistant_text)[:500]= '主题：为什么人会打喷嚏\n请写一段 300 到 400 字的无厘头科普草稿。'
DIAGNOSTIC: first 200 chars=
主题：为什么人会打喷嚏
请写一段 300 到 400 字的无厘头科普草稿。
DIAGNOSTIC: ords (first 80)= [20027, 39064, 65306, 20026, 20160, 20040, 20154, 20250, 25171, 21943, 22159, 10, 3

In [ ]:
import os
from openai import OpenAI
from colorama import Fore

from camel.societies import RolePlaying
from camel.utils import print_text_animated
from camel.models import OpenAIModel


# ========= 你的初始化（原封不动，注意 base_url） =========
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError(
        "OPENAI_API_KEY not set in environment. Please set it before running this notebook."
    )

base_url = os.getenv("OPENAI_API_BASE", "https://xiaoai.plus/v1")
model_name = os.getenv("OPENAI_MODEL", "gpt-5-mini")

client = OpenAI(api_key=api_key, base_url=base_url)


# ========= 把你现成的 client 注入 CAMEL（不重新建 client，不换 base_url） =========
# 这是“适配器”，不是另一套初始化
model = OpenAIModel(
    model_type=model_name,
    client=client,
    model_config_dict=None,  # 不额外配置温度、max_tokens，走 CAMEL 默认
)


# ========= 更强的 prompts：带评分、硬约束、禁止老套比喻 =========
assistant_prompt = """
你是无厘头科普写手，目标是写出能发表的短文。

硬约束
1 科学事实必须准确，不能编造机制与因果。
2 全文五段，总字数300到400字。
3 每段必须有一个清晰的信息点，不允许空搞笑。
4 每段至少出现一个具体名词或动作描写，避免抽象空话。

风格约束
1 无厘头来自比喻与拟人，不来自胡编科学。
2 禁止使用“机场”“消防广播”“求救信号”“入侵者”这些现成比喻与表达。
3 至少使用两种不同类型的笑点：
  a 视觉化比喻
  b 反差吐槽或拟人对话

输出
只输出正文，不输出提纲，不输出解释。
""".strip()

user_prompt = """
你是科学审稿人兼文字编辑。

任务分两部分
A 事实核查
- 用一句话指出是否存在事实风险。
- 如有风险，指出具体句子并给出替换句。

B 文学与可读性升级
- 给出0到10分评分，四项各占2.5分：
  1 科学准确
  2 信息密度
  3 笑点与新鲜感
  4 语言节奏与可读性
- 必须提出3条可执行改写指令，每条指令格式：
  指令：……
  目的：……
  替换示例：给出1句可直接替换的句子

限制
不要重复上轮相同的指令。
""".strip()


# ========= 更有效的 task_prompt：每轮只攻一个维度 =========
task_prompt = """
写一篇无厘头科普短文，主题：为什么人会打喷嚏。
要求中文，五段，总字数300到400字，科学事实准确。
必须覆盖：刺激物，鼻腔黏膜受体，三叉神经，脑干喷嚏中枢，喷嚏的防御意义。

流程
1 第一版直接完成成文稿。
2 之后每轮只做一种改进，按顺序：
  第1轮 优化机制段，压缩但更清楚。
  第2轮 更新比喻体系，必须换一套意象，不得复用旧比喻。
  第3轮 优化节奏与结尾，读完要有记忆点。
""".strip()


def run(topic: str = "为什么人会打喷嚏", rounds: int = 3) -> None:
    rp = RolePlaying(
        assistant_role_name="无厘头科普写手",
        user_role_name="科学审稿人",
        assistant_role_prompt=assistant_prompt,
        user_role_prompt=user_prompt,
        assistant_agent_kwargs={"model": model},
        user_agent_kwargs={"model": model},
        task_prompt=task_prompt.replace("为什么人会打喷嚏", topic),
        output_language="Chinese",
        with_task_specify=False,
        with_critic_in_the_loop=False,
    )

    print(Fore.YELLOW + "Task:\n" + rp.task_prompt + "\n")

    # init_chat 返回 Message 对象或一对 response，按 CAMEL 标准用法传递 msg
    input_msg = rp.init_chat()

    # 第0轮：生成第一版正文
    assistant_response, user_response = rp.step(input_msg)

    print_text_animated(Fore.BLUE + "科学审稿人:\n\n" + user_response.msg.content + "\n")
    print_text_animated(Fore.GREEN + "无厘头写手:\n\n" + assistant_response.msg.content + "\n")

    # 后续 rounds：每轮一次审稿一次重写
    input_msg = assistant_response.msg
    for i in range(rounds):
        assistant_response, user_response = rp.step(input_msg)
        if user_response.terminated or assistant_response.terminated:
            break

        print_text_animated(Fore.BLUE + f"第{i+1}轮审稿:\n\n{user_response.msg.content}\n")

        assistant_response, user_response2 = rp.step(user_response.msg)
        if assistant_response.terminated:
            break

        print_text_animated(Fore.GREEN + f"第{i+1}轮重写:\n\n{assistant_response.msg.content}\n")

        input_msg = assistant_response.msg

    print_text_animated(Fore.YELLOW + "\n最终版本:\n\n" + assistant_response.msg.content + "\n")


if __name__ == "__main__":
    run(topic="为什么人会打喷嚏", rounds=3)


Task:
写一篇无厘头科普文章，主题是：为什么人会打喷嚏。要求中文，300 到 400 字，允许离谱比喻，但科学事实不能错。

科学审稿人:

Instruction: 请先给出文章的结构提纲，并列出每段落的核心句子草案。提纲应覆盖：打喷嚏的生理机制（刺激物、鼻腔黏膜受体、三叉神经、脑干喷嚏中枢）、喷嚏在生理防御中的意义、以及允许使用离谱比喻的创意段落。文章总字数控制在300到400字之间，且科学事实准确。 
Input: None
无厘头写手:

Solution:  
结构提纲（5段）与每段核心句子草案如下，便于后续在总字数300-400字范围内完成成文稿；风格以无厘头科普为主，核心科学事实保持准确。

一、结构提纲
- 第一段：引子，点题“为什么人会打喷嚏”，以趣味设问或小故事引入。核心要求：激发好奇，奠定喷嚏是生理防御机制的主题。
- 第二段：打喷嚏的生理机制。覆盖刺激物、鼻腔黏膜受体、三叉神经、脑干喷嚏中枢的传导路径。
- 第三段：喷嚏在生理防御中的意义。强调喷嚏如何清除尘粒、花粉、微生物等，保护呼吸道。
- 第四段：创意段落，允许离谱比喻。用生动但科学可核验的比喻来增强趣味性。
- 第五段：结论与科普意义。呼应前文，给出日常生活中的科学提示与尊重身体防御的态度。

二、每段落的核心句子草案
- 第一段核心句子草案：打喷嚏不是偶发的口鼻爆炸，而是鼻腔对侵入者发出的紧急求救信号。
- 第二段核心句子草案：刺激物激活鼻腔黏膜感受器，信息经三叉神经传入脑干喷嚏中枢，指令呼吸肌群和发声器官共同发动喷出。
- 第三段核心句子草案：喷嚏的主要功能是把尘粒、花粉、微生物等异物迅速清除出鼻腔，阻断进一步进入呼吸道。
- 第四段核心句子草案：离谱比喻段落：鼻腔如同繁忙的机场，喷嚏则是一架临时起飞的超音速清洁客机，把外来粒子送回大气层。
- 第五段核心句子草案：理解喷嚏的生理机制有助于我们在日常生活中保护鼻腔健康，坦然对待这位“鼻腔卫士”的自我防卫。

三、实施与写作要点（可参考，便于控制总字数在300-400字之间）
- 字数分配建议（总目标约300-400字）：第一段60-70字，第二段80-100字，第三段60-80字，第四段60-90字，第五段40-60字；五段合计约300-400字。
- 语言风格：保持科学准确的同时，适度使用幽默、拟人化和离谱比喻，避免误导性细节